In [2]:
import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

# conn = connector.connect(
#     "semiotic-pact-388619:us-central1:tradingbot",
#     "pg8000",
#     user="postgres",
#     password="GuruSat.3",
#     db="criptos"
# )

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

In [8]:
with connection.connect() as conn:
    df = conn.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public' AND table_type='BASE TABLE'")).mappings().all()
pairs = pd.DataFrame(df)
pairs.columns = ['pairs']


path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/streaming/'

for pair in pairs['pairs']:
    with connection.connect() as conn:
        df = conn.execute(text(""" SELECT * FROM "{pair}" """.format(pair=pair)))
    df = pd.DataFrame(df)
    df.to_csv(
        path+pair,
        mode = 'w',
        index=False
    )